# 1. Định nghĩa bài toán
---

- Lunar Lander là một bài toán học tăng cường, cụ thể là 1 bài toán Quyết định Markov (MDP) rời rạc.
$$M=\{S,A,P,R,\gamma\}$$
- Trong đó:
##### Không gian trạng thái ($S$)
- Theo định nghĩa từ Gymnasium, không gian trạng thái rời rạc $S \subset R^8$ 
- Mỗi trạng thái $s\in S$ là vector gồm 8 giá trị:
$$s=[x,y,v_{x},v_{y},\theta,\omega,c_{l},c_{r}]$$
- Với:
	- $x,y:$ toạ độ của con tàu theo trục $x,y$
	- $v_{x},v_{y}:$ vận tốc của con tàu theo các phương $x,y$
	- $\theta:$ góc quay của con tàu
	- $\omega:$ vận tốc quay của con tàu
	- $c_{l},c_{r}:$ biến nhị phân thể hiện sự tiếp đất của hai chân $left,right$
##### Không gian hành động ($A$)
- Theo định nghĩa từ Gymnasium, không gian hành động $A$ rời rạc gồm các hành động:
	- $a=0:$ không làm gì
	- $a=1:$ bật động cơ chính
	- $a=2:$ bật động cơ bên trái
	- $a=3:$ bật động cơ bên phải
##### Xác suất chuyển trạng thái ($P$)
##### Hàm phần thưởng ($R$)
- Theo định nghĩa từ Gymnasium, phần thưởng nhận được sau khi chuyển từ trạng thái $s$ sang $s'$ là: $$R(s,s')=\begin{cases}
+10p \text{ với mỗi chân tiếp xúc} \\
-0.03p \text{ với mỗi động cơ hai bên bật} \\
-0.3p \text{ với động cơ chính bật} \\
-100p \text{ với hạ cánh không thành công} \\
+100p \text{ với hạ cánh thành công} \\
\text{càng tăng/giảm khi tàu càng xa/gần bãi đáp} \\
\text{càng tăng/giảm khi tốc độ tàu càng chậm/nhanh} \\
\text{càng giảm khi góc nghiêng tàu càng lớn} \\
\end{cases}$$
##### Hệ số chiết khấu (discount factor: $\gamma$)

#### 2. Trạng thái kết thúc
- Tàu hạ cánh thành công:
	- Cả 2 chân đều chạm đất: $c_{l}=c_{r}=1$
	- Vận tốc rơi nhỏ và góc nghiêng nhỏ
	- Tổng $\text{reward} \geq 200$
- Tàu rơi hoặc lật:
	- Chạm đất với vận tốc lớn hoặc góc nghiêng lớn
	- Rơi ra khỏi màn hinh
	- Tổng $\text{reward} \leq -100$
- Vượt quá số bước tối đa:
	- Mỗi episode có số step tối đa là 1000
	- Nếu không hạ cánh hay rơi trong khoảng số bước này, episode sẽ bị huỷ
#### 3. Mục tiêu bài toán
- Tìm chính sách tối ưu $\pi^*:S\to A$ sao cho kỳ vọng của tổng phần thưởng được tối đa
$$\pi^*=argmax_{\pi}E_{\pi}\left[ \sum_{t=0}^{\infty}\gamma^t.R(s_{t},a_{t},s_{t+1}) \right]$$

# 2. Cài đặt tổng quan
---

## 2.1. Thư viện, các hàm xử lí chung và môi trường LunarLander

#### a. Thư viện và cài đặt

In [1]:
# !pip install gymnasium
# !pip install swig
# !pip install gymnasium[box2d]
# !pip install torch
# !pip install numpy
# !pip install matplotlib
# !pip install tqdm
# !pip install moviepy

In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from collections import deque
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import random
import glob
from IPython.display import Video
from tqdm import tqdm

#### b. Các hàm plot và đường MA

In [ ]:
def plot_rewards(rewards, moving_average_window=50):
    # Calculate moving average
    moving_avg = np.convolve(rewards, np.ones(moving_average_window)/moving_average_window, mode='valid')
    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(rewards, label='Rewards', alpha=0.5)
    plt.plot(np.arange(moving_average_window - 1, len(rewards)), moving_avg, label='Moving Average', color='red')
    plt.title('Rewards and Moving Average')
    plt.xlabel('Episode')
    plt.ylabel('Reward')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def plot_losses(losses, moving_average_window=50):
    # Calculate moving average
    moving_avg = np.convolve(losses, np.ones(moving_average_window)/moving_average_window, mode='valid')
    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(losses, label='Losses', alpha=0.5)
    plt.plot(np.arange(moving_average_window - 1, len(losses)), moving_avg, label='Moving Average', color='red')
    plt.title('Losses and Moving Average')
    plt.xlabel('Episode')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
def plot_fuel_usage(fuel_usage, moving_average_window=50):
    # Calculate moving average
    moving_avg = np.convolve(fuel_usage, np.ones(moving_average_window)/moving_average_window, mode='valid')
    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(fuel_usage, label='Fuel Usage', alpha=0.5)
    plt.plot(np.arange(moving_average_window - 1, len(fuel_usage)), moving_avg, label='Moving Average', color='red')
    plt.title('Fuel Usage and Moving Average')
    plt.xlabel('Episode')
    plt.ylabel('Fuel Usage')
    plt.legend()
    plt.grid()
    plt.show()

#### c. Môi trường LunarLander

In [ ]:
env = gym.make("LunarLander-v3")
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

In [ ]:
# Số chiều của không gian trạng thái và hành động
print(f"State Dimension: {state_dim}, Action Dimension: {action_dim}")

State Dimension: 8, Action Dimension: 4


In [ ]:
# Nhiên liệu tiêu thụ cho mỗi hành động
FUEL_COST = {0: 0.0, 1: 0.3, 2: 0.03, 3: 0.03}

In [ ]:
# Sử dụng GPU nếu có
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## 2.2. Random action

Trước hết, ta sẽ khởi tạo một lớp có tên `RandomTrain` để kiểm tra các hàm xử lý chung và đảm bảo rằng các chức năng cơ bản của môi trường hoạt động đúng như mong đợi. Việc kiểm tra này giúp phát hiện sớm các lỗi tiềm ẩn trước khi áp dụng các thuật toán huấn luyện phức tạp hơn. Đồng thời, quá trình này cũng cho phép ghi lại và hiển thị một video mẫu minh họa cách tác nhân hoạt động khi hành động được chọn một cách ngẫu nhiên.

In [ ]:
class RandomTrain:
    def __init__(self, env, episodes):
        self.env = env
        self.episodes = episodes
        self.rewards = []
        self.losses = []
        self.fuel_usage = []
        self.model = None

    def train(self):
        for episode in tqdm(range(self.episodes)):
            state, _ = self.env.reset()
            total_reward = 0
            total_fuel = 0
            done = False

            while not done:
                action = self.get_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                total_reward += reward
                total_fuel += FUEL_COST[action]
                state = next_state
                done = terminated or truncated

            self.rewards.append(total_reward)
            self.fuel_usage.append(total_fuel)
            #print(f"Episode {episode + 1}/{self.episodes}, Reward: {total_reward}, Fuel Used: {total_fuel}")
            self.env.close()
        return self.rewards, self.losses, self.fuel_usage
    
    def plot_results(self):
        plot_rewards(self.rewards)
        plot_fuel_usage(self.fuel_usage)
        
    def get_action(self, state):
        return np.random.randint(0, action_dim)

    def display_sample_video(self, sample_episode=5):
        video_render = gym.make("LunarLander-v3", render_mode="rgb_array")
        video_render = RecordVideo(video_render, "video-random", episode_trigger=lambda x: True)
        for _ in range(sample_episode):
            state, _ = video_render.reset()
            done = False
            while not done:
                action = self.get_action(state)
                state, reward, terminated, truncated, _ = video_render.step(action)
                done = terminated or truncated
            video_render.close()
        video_files = glob.glob("video-random/*.mp4")
        return Video(video_files[-1])
    
    def test_model(self, episodes=500):
        total_success = 0
        testing = gym.make("LunarLander-v3")
        
        for episode in tqdm(range(episodes)):
            state, _ = testing.reset()
            done = False
            total_reward = 0
            self.epsilon = 0
            
            while not done:
                action = self.get_action(state)
                state, reward, terminated, truncated, _ = testing.step(action)
                total_reward += reward
                done = terminated or truncated
            if total_reward >= 200:
                total_success += 1
        print(f"Success: {total_success}/{episodes} | Success Rate: {total_success / episodes * 100:.2f}%")
        testing.close()

### Kết quả

In [ ]:
# random_train = RandomTrain(env, episodes=500)
# random_train.display_sample_video(sample_episode=1)

<video autoplay controls src="...">
  The “video” tag is not supported by your browser.
</video>

# 3. Q-Learning sử dụng Q-Table
---

### a. Giới thiệu

- Q-Learning là một thuật toán học tăng cường không mô hình sử dụng bảng Q-Table để lưu trữ và cập nhật giá trị kỳ vọng của mỗi hành động tại từng trạng thái.
- Q-Table là một bảng trong đó mỗi phần tử `Q(s, a)` biểu thị giá trị kỳ vọng của việc thực hiện hành động `a` tại trạng thái `s` và sau đó tuân theo chính sách tối ưu.
- Vì Q-learning chỉ áp dụng được cho không gian trạng thái `rời rạc`, nên cần lượng tử hóa (discretize) không gian trạng thái của LunarLander `(liên tục)` thành các bins.

### b. Công thức toán học

- Q-Table được cập nhật theo công thức Bellman:
$$
Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma \cdot \max_{a'} Q(s', a') - Q(s, a) \right]
$$

Trong đó:

- $s$: trạng thái hiện tại (*current state*)
- $a$: hành động đã chọn (*action taken*)
- $r$: phần thưởng nhận được (*reward received*)
- $s'$: trạng thái tiếp theo (*next state*)
- $\alpha \in (0, 1)$: hệ số học (*learning rate*)
- $\gamma \in (0, 1)$: hệ số chiết khấu (*discount factor*)
- $\max_{a'} Q(s', a')$: giá trị Q tối đa tại trạng thái tiếp theo

### c. Triển khai code

Các hàm chính xử lí thuật toán:

`__init__`

Hàm khởi tạo các tham số cần thiết để huấn luyện:
- Các tham số của thuật toán.
- **bins_size**: Số lượng đoạn chia mỗi chiều trong không gian trạng thái (discretization).
- **state_bins**: Mỗi chiều của trạng thái được chia nhỏ thành các đoạn (bin).
- **q_table**: Q-table 9 chiều chứa giá trị Q cho mỗi tổ hợp trạng thái và hành động. Kích thước:
$$(\text{bins\_size})^6*2*2*\text{số hành động}$$

`get_state_bin(state)`

- Lượng tử hóa (discretize) trạng thái liên tục thành các chỉ số bin rời rạc.
- Dùng np.digitize để xác định chỉ số bin tương ứng.
- Đảm bảo chỉ số nằm trong phạm vi hợp lệ.

`get_action(state)`

Triển khai chính sách ε-greedy:
- Với xác suất ε, chọn hành động ngẫu nhiên (khám phá).
- Ngược lại, chọn hành động có giá trị Q lớn nhất từ Q-table.

`train()`

Hàm huấn luyện tác tử trong nhiều vòng lặp (episode):

1. Mỗi bước:
   - Chọn hành động bằng `get_action(state)`
   - Tương tác với môi trường, nhận phần thưởng $r$
   - Xác định trạng thái kế tiếp $s'$
   - Tính toán sai số TD:

     $$
     \delta = \left[ r + \gamma \cdot \max_{a'} Q(s', a') \right] - Q(s, a)
     $$

   - Cập nhật giá trị Q:

     $$
     Q(s, a) \leftarrow Q(s, a) + \alpha \cdot \delta
     $$

2. Sau mỗi vòng lặp, giảm $\varepsilon$ theo công thức:

   $$
   \varepsilon \leftarrow \max(\varepsilon_{\text{min}}, \varepsilon \cdot \varepsilon_{\text{decay}})
   $$

3. Ghi nhận tổng phần thưởng và mức tiêu hao nhiên liệu.



In [ ]:
class QTable:
    def __init__(self, env, episodes, alpha, gamma, epsilon, epsilon_decay, min_epsilon, bins_size):
        ''' 
        Khởi tạo các tham số cho Q-table
        '''
        self.env = env # Môi trường bình thường
        self.episodes = episodes # Số tập huấn luyện
        self.alpha = alpha # Tốc độ học (lr)
        self.gamma = gamma # Discount factor
        self.epsilon = epsilon # Tỉ lệ khám phá
        self.epsilon_decay = epsilon_decay # Tỉ lệ giảm epsilon
        self.min_epsilon = min_epsilon # Giá trị epsilon tối thiểu
        self.bins_size = bins_size # Kích thước của các bin
        self.rewards = []
        self.fuel_usage = []

        # Khởi tạo state_bins là một danh sách chứa các khoảng giá trị cho từng trạng thái
        self.state_bins = [
            np.linspace(-1.5, 1.5, bins_size), # X
            np.linspace(-0.5, 1.5, bins_size), # Y
            np.linspace(-2, 2, bins_size), # X vận tốc
            np.linspace(-2, 2, bins_size), # Y vận tốc
            np.linspace(-3.14, 3.14, bins_size), # Góc
            np.linspace(-5, 5, bins_size), # Góc vận tốc
            np.array([0, 1]), # Chân trái tiếp đất
            np.array([0, 1]), # Chân phải tiếp đất
        ]

        # Khởi tạo Q-table với kích thưởc của các bin
        self.q_table = np.zeros((bins_size, bins_size, bins_size, bins_size,
                                 bins_size, bins_size, 2, 2, action_dim))


    def get_state_bin(self, state):
        '''
        Chuyển đổi trạng thái liên tục thành trạng thái rời rạc 
        bằng cách sử dụng các khoảng giá trị đã định nghĩa
        '''
        state_bins = []
        for i in range(len(state)):
            bin_index = np.digitize(state[i], self.state_bins[i]) - 1
            bin_index = min(max(bin_index, 0), len(self.state_bins[i]) - 1)
            state_bins.append(bin_index)
        return tuple(state_bins)

    def get_action(self, state):
        '''
        Chọn hành động dựa trên epsilon-greedy policy
        '''
        
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, action_dim)
        else:
            return np.argmax(self.q_table[self.get_state_bin(state)])

    def train(self):
        for episode in tqdm(range(self.episodes)):
            state, _ = self.env.reset()
            total_reward = 0
            total_fuel = 0
            done = False

            while not done:
                # lấy hành động
                action = self.get_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                total_reward += reward
                total_fuel += FUEL_COST[action]

                # Lấy trạng thái rời rạc
                state_bin = self.get_state_bin(state)
                # Lấy trạng thái rời rạc tiếp theo
                next_state_bin = self.get_state_bin(next_state)

                # Lấy hành động tốt nhất tiếp theo
                best_next_action = np.argmax(self.q_table[next_state_bin])
                
                # Tính toán giá trị TD target và TD error
                td_target = reward + self.gamma * self.q_table[next_state_bin][best_next_action]
                td_error = td_target - self.q_table[state_bin][action]
                
                # Cập nhật Q-value
                self.q_table[state_bin][action] += self.alpha * td_error
                
                state = next_state
                done = terminated or truncated

            self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)
            self.rewards.append(total_reward)
            self.fuel_usage.append(total_fuel)

        return self.rewards, [], self.fuel_usage
    
    def plot_results(self):
        '''
        Vẽ biểu đồ kết quả huấn luyện
        '''
        plot_rewards(self.rewards)
        plot_fuel_usage(self.fuel_usage)
    
    def display_sample_video(self, sample_episode=5):
        '''
        Hiển thị video mẫu
        '''
        video_render = gym.make("LunarLander-v3", render_mode="rgb_array")
        video_render = RecordVideo(video_render, "video-q-table", episode_trigger=lambda x: True)
        for _ in range(sample_episode):
            state, _ = video_render.reset()
            done = False
            while not done:
                action = self.get_action(state)
                state, reward, terminated, truncated, _ = video_render.step(action)
                done = terminated or truncated
            video_render.close()
        video_files = glob.glob("video-q-table/*.mp4")
        return Video(video_files[-1])
    
    def test_model(self, episodes=500):
        '''
        Kiểm tra mô hình đã huấn luyện
        '''
        total_success = 0
        testing = gym.make("LunarLander-v3")
        
        for episode in tqdm(range(episodes)):
            state, _ = testing.reset()
            done = False
            total_reward = 0
            self.epsilon = 0
            
            while not done:
                action = self.get_action(state)
                state, reward, terminated, truncated, _ = testing.step(action)
                total_reward += reward
                done = terminated or truncated
            if total_reward >= 200:
                total_success += 1
        print(f"Success: {total_success}/{episodes} | Success Rate: {total_success / episodes * 100:.2f}%")
        testing.close()

### d. Kết quả chính

In [ ]:
# train = QTable(env, 
#                episodes=50000, 
#                alpha=0.01, 
#                gamma=0.99, 
#                epsilon=1.0, 
#                epsilon_decay=0.9995, 
#                min_epsilon=0.01, 
#                bins_size=8)

# rewards, losses, fuel_usage = train.train()

100%|██████████| 50000/50000 [31:03<00:00, 26.84it/s]

In [ ]:
# train.plot_results()

<div style="display: flex; height: 300px;">

  <div style="flex: 1; overflow: hidden;">
    <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/q-table.png?raw=true" style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh trái"/>
  </div>

  <div style="flex: 1; overflow: hidden;">
    <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/q-table1.png?raw=true" style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh phải"/>
  </div>

</div>


In [ ]:
# train.test_model(episodes=500)

100%|██████████| 500/500 [00:10<00:00, 49.98it/s]
Success: 61/500 | Success Rate: 12.20%

In [ ]:
# train.display_sample_video(sample_episode=5)

<video autoplay controls src="...">
  The “video” tag is not supported by your browser.
</video>

### e. Kết luận

- Quá trình huấn luyện 50.000 episodes mất 31p, với tốc độ khoảng 26.84 it/s (tốc độ khá cao cho thấy khả năng rằng mô hình không học được nhiều). Đây là một khoảng thời gian khá dài cho một môi trường giả lập như Lunar Lander, đặc biệt khi so sánh với hiệu quả đạt được.
- Tỉ lệ thành công sau 500 lần kiểm tra là 12.20% (61/500). Đây là một tỉ lệ rất thấp, cho thấy mô hình Q-Table chưa học được cách hạ cánh Lunar Lander một cách hiệu quả.

- Nguyên nhân có thể: 
    - Không gian trạng thái lớn: Lunar Lander có không gian trạng thái liên tục. Việc chia nhỏ (binning) tạo ra một Q-table quá lớn và thưa thớt, 50.000 episodes không đủ để khám phá. bins_size=8 có thể quá thô, gộp các trạng thái quan trọng.
    - Hiện tượng tràn RAM khi để bins_size quá cao
    - Mô hình Q-learning với Q-Table không đủ khả năng tổng quát hoá.
    - Tốc độ học (alpha) thấp: alpha=0.01 làm chậm quá trình hội tụ trong không gian trạng thái lớn.
    - Suy giảm epsilon nhanh: epsilon_decay=0.9995 khiến quá trình khám phá dừng lại quá sớm, trước khi mô hình đủ kinh nghiệm.


**$\implies$ Giải quyết:** Q-Table gặp khó khăn lớn với không gian trạng thái liên tục hoặc quá lớn. Đối với các bài toán như Lunar Lander, các thuật toán học tăng cường dựa trên mạng nơ-ron như Deep Q-Networks (DQN) sẽ cho hiệu quả tốt hơn, tránh hiện tượng tràn RAM.

# 4. Deep Q-Network
---

### a. Giới thiệu

DQN là thuật toán tăng cường học sâu, nhằm giải quyết bài toán ra quyết định trong môi trường rời rạc, kết hợp:
- Mạng Neuron học sâu
- Q-Learning cổ điển

Khác với Q-learning truyền thống chỉ sử dụng Q-Table để lưu trữ giá trị hành động, DQN sử dụng một mạng nơ-ron để xấp xỉ hàm Q, giúp mở rộng khả năng áp dụng sang các môi trường có không gian trạng thái lớn hoặc liên tục.

Thuật toán DQN hoạt động bằng cách cho tác nhân tương tác với môi trường, thu thập kinh nghiệm dưới dạng các bộ dữ liệu (trạng thái, hành động, phần thưởng, trạng thái kế tiếp), và sử dụng các dữ liệu này để huấn luyện mạng nơ-ron sao cho đầu ra của nó gần đúng với giá trị Q tối ưu. Mục tiêu của DQN là tìm một hàm Q* sao cho tại mỗi trạng thái, hành động được chọn sẽ tối đa hóa tổng phần thưởng trong dài hạn.

### b. Công thức toán học

Tại mỗi bước huấn luyện, DQN tối thiểu hóa hàm mất mát sau:

$$
\mathcal{L}(\theta) = \mathbb{E}_{(s, a, r, s') \sim \mathcal{D}} \left[ \left( y - Q(s, a; \theta) \right)^2 \right]
$$

Trong đó:

- $ \theta $ là tham số của mạng Q hiện tại.  
- $ \mathcal{D} $ là replay buffer (bộ nhớ kinh nghiệm) lưu trữ các trạng thái của môi trường.
- $ y $ là giá trị mục tiêu, xác định bởi công thức: 

$$
y = r + \gamma \cdot \max_{a'} Q(s', a'; \theta^{-})
$$

- $ Q(s, a; \theta) $: giá trị Q được ước lượng bởi mạng hiện tại.  
- $ \theta^{-} $: tham số của mạng Q mục tiêu (target network), được cập nhật định kỳ từ $ \theta $.  
- $ \gamma \in [0, 1] $: hệ số chiết khấu phần thưởng tương lai.


Việc sử dụng `Replay Buffer` và `Target Q-Net` là hai cải tiến chính giúp DQN ổn định hơn và tránh hiện tượng học lệch, từ đó cho kết quả huấn luyện hiệu quả hơn so với các phương pháp Q-learning truyền thống.




### c. Triển khai code

Lớp `DQNetwork` là một mạng nơ-ron sâu (deep neural network) được sử dụng để ước lượng hàm Q trong thuật toán Deep Q-Learning (DQN).

Mạng này bao gồm:

- Một lớp đầu vào với kích thước bằng số chiều trạng thái (`state_dim`).
- Một lớp ẩn, mỗi lớp gồm 256 nút và sử dụng hàm kích hoạt ReLU.
- Một lớp đầu ra với kích thước bằng số hành động (`action_dim`), trả về giá trị Q tương ứng với từng hành động.

In [ ]:
class DQNetwork(nn.Module):
    def __init__(self, state_dim, action_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(state_dim, 256) ,
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, action_dim)
        )
    def forward(self, x):
        return self.model(x)

Lớp `DQNTrain` chịu trách nhiệm huấn luyện agent sử dụng thuật toán Deep Q-Network (DQN) trên môi trường như LunarLander-v3. Nó bao gồm các thành phần chính như sau:

**Khởi tạo mô hình**
- Khởi tạo mạng Q chính (`q_net`) và mạng Q mục tiêu (`target_net`).
- Khởi tạo replay buffer để lưu trữ kinh nghiệm.
- Chọn thuật toán tối ưu hóa (````adam````, RMSprop hoặc SGD).
- Thiết lập các siêu tham số như epsilon, gamma, batch size, số tập huấn luyện,...

**Chọn hành động (`select_action`)**
- Dựa trên chính sách ε-greedy: chọn ngẫu nhiên hoặc hành động tốt nhất theo Q hiện tại. Chính sách này kết hợp giữa hành động ngẫu nhiên và hành động tối ưu:
    - Với xác suất ε (epsilon), chọn hành động ngẫu nhiên.

    - Với xác suất 1 - ε, chọn hành động tốt nhất (hành động có giá trị Q cao nhất).

- Mục đích của ε-greedy là khuyến khích việc khám phá không gian trạng thái trong giai đoạn đầu và sau đó chuyển sang khai thác những hành động tối ưu đã học được khi epsilon giảm dần.

**Huấn luyện từng bước (`train_step`)**

- Lấy minibatch từ replay buffer.
- Tính toán giá trị Q hiện tại và Q mục tiêu:

1. Q-value hiện tại (lấy từ mạng `q_net`):

$$Q_{predict}=Q(s,a,\theta)$$

2. Q-target (lấy từ mạng `target_net`) cập nhật bằng Bellman target:

$$Q_{target}​=r+γ⋅a'.max​_{a'}Q(s',a';θ^{-})⋅(1−done)$$

`?` Tại sao phải có mạng target_net ở đây: Nếu dùng cùng 1 mạng Q để tính cả Q-value cả Q-target thì Q-target cũng thay đổi theo $\theta$ khi cập nhật.

=> Quá trình học sẽ phải đuổi theo một Q-target thay đổi liên tục => không hiệu quả

=> `Giải pháp`: tạo thêm 1 mạng target_net được cập nhật sau mỗi k episodes để mô hình đạt hiệu quả cao nhất.

- Tối ưu hóa bằng hàm mất mát MSE để cập nhật mạng Q.

**Vòng lặp huấn luyện (`train`)**
- Lặp qua các episode, cập nhật mạng target_net định kỳ.
- Theo dõi reward, loss và mức tiêu hao nhiên liệu qua từng tập.

Các chú thích chi tiết ở trong code sau:


In [ ]:
class DQNTrain:
    def __init__(self, env, optimizer, epsilon, min_epsilon, decay, gamma, batch_size, episodes, target_update_freq, memory_size, learning_rate):
        '''
        - Khởi tạo các tham số cho DQN
        - Khởi tạo optimizer (adam, RMSprop, SGD)
        - Khởi tạo mạng Q và mạng Q mục tiêu
        - Khởi tạo bộ nhớ (Replay Buffer) để lưu trữ các trạng thái của môi trường
        - Khởi tạo các biến để theo dõi phần thưởng, tổn thất và mức tiêu thụ nhiên liệu
        '''
        self.env = env # Môi trường
        self.episodes = episodes # Số lượng tập huấn luyện
        self.epsilon = epsilon # Giá trị epsilon ban đầu
        self.min_epsilon = min_epsilon # Giá trị epsilon tối thiểu
        self.decay = decay # Hệ số giảm epsilon
        self.gamma = gamma # Hệ số giảm giá cho phần thưởng
        self.batch_size = batch_size # Kích thước batch cho việc huấn luyện
        self.target_update_freq = target_update_freq # Tần suất cập nhật mạng Q mục tiêu
        self.memory = deque(maxlen=memory_size) # Bộ nhớ để lưu trữ các trạng thái (replay buffer)
        self.fuel_usage = [] # Danh sách để lưu trữ mức tiêu thụ nhiên liệu
        self.rewards = [] # Danh sách để lưu trữ phần thưởng
        self.losses = [] # Danh sách để lưu trữ loss
        
        self.q_net = DQNetwork(state_dim, action_dim).to(device)
        self.target_net = DQNetwork(state_dim, action_dim).to(device)
        self.target_net.load_state_dict(self.q_net.state_dict())
        self.target_net.eval()
        
        if optimizer == 'adam':
            self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=learning_rate)
        elif optimizer == 'rmsprop':
            self.optimizer = torch.optim.RMSprop(self.q_net.parameters(), lr=learning_rate)
        elif optimizer == 'sgd':
            self.optimizer = torch.optim.SGD(self.q_net.parameters(), lr=learning_rate)
        
        
    def select_action(self, state):
        '''
        - Chọn hành động dựa trên epsilon-greedy
        + Nếu giá trị ngẫu nhiên nhỏ hơn epsilon, chọn hành động ngẫu nhiên
        + Ngược lại, chọn hành động có giá trị Q lớn nhất từ mạng Q
        - Trả về chỉ số của hành động được chọn 
        '''
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, action_dim)
        else:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
                return self.q_net(state_tensor).argmax().item()
            

    def train_step(self):
        ''''
        - Huấn luyện mạng Q bằng cách lấy mẫu ngẫu nhiên từ bộ nhớ, nếu bộ nhớ lớn hơn kích thước batch
        - Chuyển đổi các trạng thái, hành động, phần thưởng, trạng thái tiếp theo và done thành tensor
        - Tính toán giá trị Q hiện tại và giá trị Q mục tiêu
        - Tính toán loss bằng cách sử dụng hàm mất mát MSE
        - Cập nhật trọng số của mạng Q bằng cách sử dụng thuật toán tối ưu hóa
        - Trả về giá trị loss
        '''
        
        if len(self.memory) < self.batch_size:
            return 0
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = np.array(states)
        actions = np.array(actions)
        rewards = np.array(rewards)
        next_states = np.array(next_states)
        dones = np.array(dones)

        states = torch.FloatTensor(np.array(states)).to(device)
        actions = torch.LongTensor(actions).unsqueeze(1).to(device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
        next_states = torch.FloatTensor(np.array(next_states)).to(device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(device)

        q_values = self.q_net(states).gather(1, actions)

        with torch.no_grad():
            max_next_q = self.target_net(next_states).max(1)[0].unsqueeze(1)
            targets = rewards + self.gamma * max_next_q * (1 - dones)

        loss = nn.MSELoss()(q_values, targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def train(self):
        '''
        - Huấn luyện DQN bằng cách lặp qua số lượng tập huấn luyện
        - Trong mỗi tập, khởi tạo trạng thái và phần thưởng
        - Lặp qua các bước trong mỗi tập cho đến khi hoàn thành
        - Chọn hành động bằng cách sử dụng hàm select_action
        - Thực hiện hành động và nhận trạng thái tiếp theo, phần thưởng và trạng thái hoàn thành
        - Lưu trữ trạng thái vào bộ nhớ
        - Tính toán loss bằng cách sử dụng hàm train_step
        - Cập nhật mạng Q mục tiêu theo tần suất đã chỉ định
        - Lưu trữ phần thưởng, loss và mức tiêu thụ nhiên liệu vào danh sách
        - Trả về danh sách phần thưởng, loss và mức tiêu thụ nhiên liệu
        '''
        
        pbar = tqdm(range(self.episodes), desc="Training DQN")
        
        for ep in pbar:
            state, _ = self.env.reset()
            total_reward = 0
            total_loss = 0
            total_fuel = 0
            done = False

            while not done:
                action = self.select_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                self.memory.append((state, action, reward, next_state, float(done)))
                
                loss = self.train_step()
                total_loss += loss
                total_reward += reward
                total_fuel += FUEL_COST[action]
                
                state = next_state

            if ep % self.target_update_freq == 0:
                self.target_net.load_state_dict(self.q_net.state_dict())
                
            if ep % 100 == 0:
                average_reward = np.mean(self.rewards[-100:])
                print(f"Episode {ep}, Average Reward: {average_reward:.2f}")
                
            self.epsilon = max(self.min_epsilon, self.epsilon * self.decay)
            self.rewards.append(total_reward)
            self.losses.append(total_loss)
            self.fuel_usage.append(total_fuel)

            pbar.set_postfix({
                "Reward": total_reward,
                "Epsilon": self.epsilon,
                "Loss": total_loss,
                "Fuel Usage": total_fuel
            })

        return self.rewards, self.losses, self.fuel_usage


    def save_model(self, path):
        '''
        - Lưu trọng số của mạng Q vào tệp
        '''
        torch.save(self.q_net.state_dict(), path)


    def load_model(self, path):
        '''
        - Tải trọng số của mạng Q từ tệp
        - Đặt mạng Q ở chế độ đánh giá để không cập nhật trọng số trong quá trình kiểm tra
        '''
        self.q_net.load_state_dict(torch.load(path))
        self.q_net.eval()
        
        
    def get_target_action(self, state):
        ''' 
        - Chọn hành động từ mạng Q để kiểm tra (không có epsilon-greedy)
        '''
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
            return self.q_net(state_tensor).argmax().item()


    def plot_results(self):
        '''
        - Vẽ biểu đồ phần thưởng, tổn thất và mức tiêu thụ nhiên liệu
        - Sử dụng các hàm plot_rewards, plot_losses và plot_fuel_usage đã định nghĩa ở trên
        '''
        plot_rewards(self.rewards)
        plot_losses(self.losses)
        plot_fuel_usage(self.fuel_usage)
        
    
    def test_model(self, episodes=500):
        '''
        - Kiểm tra mô hình đã được huấn luyện bằng cách chạy một số tập
        - Trong mỗi tập, khởi tạo trạng thái và phần thưởng
        - Lặp qua các bước trong mỗi tập cho đến khi hoàn thành
        - Chọn hành động bằng cách sử dụng hàm get_target_action
        - Thực hiện hành động và nhận trạng thái tiếp theo, phần thưởng và trạng thái hoàn thành
        - Tính toán tổng phần thưởng
        - Nếu tổng phần thưởng lớn hơn hoặc bằng 200, tăng biến thành công
        - Trả về số lượng thành công và tỷ lệ thành công
        '''
        total_success = 0
        testing = gym.make("LunarLander-v3")
        
        for episode in tqdm(range(episodes)):
            state, _ = testing.reset()
            done = False
            total_reward = 0
            
            while not done:
                action = self.get_target_action(state)
                state, reward, terminated, truncated, _ = testing.step(action)
                total_reward += reward
                done = terminated or truncated
            if total_reward >= 200:
                total_success += 1
        print(f"Success: {total_success}/{episodes} | Success Rate: {total_success / episodes * 100:.2f}%")
        testing.close()
        
        
    def display_sample_video(self, sample_video=1):
        '''
        - Hiển thị video mẫu của mô hình đã được huấn luyện
        - Sử dụng RecordVideo để ghi lại video trong môi trường
        - Trong mỗi tập, khởi tạo trạng thái và phần thưởng
        - Lặp qua các bước trong mỗi tập cho đến khi hoàn thành
        - Chọn hành động bằng cách sử dụng hàm get_target_action
        - Thực hiện hành động và nhận trạng thái tiếp theo, phần thưởng và trạng thái hoàn thành
        - Đóng video sau khi hoàn thành
        - Trả về video cuối cùng được ghi lại
        '''
        video_render = gym.make("LunarLander-v3", render_mode="rgb_array")
        video_render = RecordVideo(video_render, "videos", episode_trigger=lambda x: True)
        
        for _ in tqdm(range(sample_video)):
            state, _ = video_render.reset()
            done = False
            while not done:
                action = self.get_target_action(state)
                state, reward, terminated, truncated, _ = video_render.step(action)
                done = terminated or truncated
            video_render.close()
        video_files = glob.glob("videos/*.mp4")
        return Video(video_files[-1])

### d. Kết quả chính

#### Kết quả với tối ưu RMSPROP

In [ ]:
trainer = DQNTrain(
    env=env,
    optimizer='rmsprop',
    epsilon=1.0,
    min_epsilon=0.01,
    decay=0.995,
    gamma=0.99,
    batch_size=128,
    episodes=2000,
    target_update_freq=10,
    memory_size=100000,
    learning_rate=1e-4
)

In [ ]:
# rewards, losses, fuel_usage = trainer.train()

Training DQN: 100%|██████████| 2000/2000 [38:25<00:00,  1.15s/it, Reward=237, Epsilon=0.01, Loss=1.84e+3, Fuel Usage=13.5]
-    Episode 0, Average Reward: nan
-    Episode 100, Average Reward: -140.42
-    Episode 200, Average Reward: -58.85  
-    Episode 300, Average Reward: -22.66
-    Episode 400, Average Reward: 33.97 
-    Episode 500, Average Reward: 32.39
-    Episode 600, Average Reward: 104.68
-    Episode 700, Average Reward: 117.25 
-    Episode 800, Average Reward: 181.27
-    Episode 900, Average Reward: 240.24
-    Episode 1000, Average Reward: 237.16
-    Episode 1100, Average Reward: 244.40
-    Episode 1200, Average Reward: 260.15 
-    Episode 1300, Average Reward: 259.16
-    Episode 1400, Average Reward: 247.33
-    Episode 1500, Average Reward: 245.86
-    Episode 1600, Average Reward: 256.02
-    Episode 1700, Average Reward: 252.03
-    Episode 1800, Average Reward: 259.45  
-    Episode 1900, Average Reward: 260.10

In [ ]:
# trainer.plot_results()

<div style="display: flex; flex-direction: column; height: 600px;">

  <!-- Hàng trên: 2 ảnh cạnh nhau -->
  <div style="flex: 1; display: flex;">
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/rmsprop.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh trái"/>
    </div>
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/rmsprop1.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh phải"/>
    </div>
  </div>

  <!-- Hàng dưới: 1 ảnh chiếm toàn bộ chiều ngang -->
  <div style="flex: 1; overflow: hidden;">
    <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/rmsprop2.png?raw=true" 
         style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh dưới"/>
  </div>

</div>


In [ ]:
# trainer.save_model("dqn_lunarlander-2k-rmsprop.pth")

In [ ]:
# trainer.load_model("dqn_lunarlander-2k-rmsprop.pth")

In [ ]:
# trainer.test_model(episodes=500)

100%|██████████| 500/500 [00:29<00:00, 17.19it/s]
Success: 482/500 | Success Rate: 96.40%

In [ ]:
# trainer.display_sample_video(sample_video=5)

<video autoplay controls src="...">
  The “video” tag is not supported by your browser.
</video>

#### Kết quả với tối ưu Adam

In [ ]:
trainer = DQNTrain(
    env=env,
    optimizer='adam',
    epsilon=1.0,
    min_epsilon=0.01,
    decay=0.995,
    gamma=0.99,
    batch_size=128,
    episodes=2000,
    target_update_freq=10,
    memory_size=100000,
    learning_rate=1e-4
)

In [ ]:
# rewards, losses, fuel_usage = trainer.train()

Training DQN: 100%|██████████| 2000/2000 [34:30<00:00,  1.04s/it, Reward=276, Epsilon=0.01, Loss=1.87e+3, Fuel Usage=14.4]  
- Episode 0, Average Reward: nan
- Episode 100, Average Reward: -119.58
- Episode 200, Average Reward: -68.83
- Episode 300, Average Reward: -16.85
- Episode 400, Average Reward: 47.01 
- Episode 500, Average Reward: 74.07  
- Episode 600, Average Reward: 152.28 
- Episode 700, Average Reward: 164.76 
- Episode 800, Average Reward: 221.37
- Episode 900, Average Reward: 233.87
- Episode 1000, Average Reward: 260.68
- Episode 1100, Average Reward: 259.33
- Episode 1200, Average Reward: 255.35
- Episode 1300, Average Reward: 266.93
- Episode 1400, Average Reward: 273.05
- Episode 1500, Average Reward: 278.39
- Episode 1600, Average Reward: 277.69
- Episode 1700, Average Reward: 267.48
- Episode 1800, Average Reward: 255.86 
- Episode 1900, Average Reward: 248.64


In [ ]:
# trainer.plot_results()

<div style="display: flex; flex-direction: column; height: 600px;">

  <!-- Hàng trên: 2 ảnh cạnh nhau -->
  <div style="flex: 1; display: flex;">
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/adam.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh trái"/>
    </div>
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/adam1.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh phải"/>
    </div>
  </div>

  <!-- Hàng dưới: 1 ảnh chiếm toàn bộ chiều ngang -->
  <div style="flex: 1; overflow: hidden;">
    <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/adam2.png?raw=true" 
         style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh dưới"/>
  </div>

</div>


In [ ]:
# trainer.save_model("dqn_lunarlander-2k-adam.pth")

In [ ]:
# trainer.load_model("dqn_lunarlander-2k-adam.pth")

In [ ]:
# trainer.test_model(episodes=500)

Success: 484/500 | Success Rate: 96.80%

In [ ]:
# trainer.display_sample_video(sample_video=5)

<video autoplay controls src="...">
  The “video” tag is not supported by your browser.
</video>

#### **So sánh**

| Tiêu chí                              | RMSprop         | Adam           |
|--------------------------------------|------------------|----------------|
| Thời gian huấn luyện                 | 38 phút 25 giây | 34 phút 30 giây |
| Reward cuối                          | 237             | 276            |
| Tỉ lệ thành công                     | 96.40%          | 96.80%         |
| Tiêu thụ nhiên liệu                  | 13.5            | 14.4           |
| Reward TB @ Episode 1000            | 237.16          | 260.68         |
| Reward TB @ Episode 1900            | 260.10          | 248.64         |

**Phân tích**
- **Thời gian huấn luyện**: Adam nhanh hơn ~4 phút, phản ánh khả năng cập nhật trọng số hiệu quả hơn.
- **Tỉ lệ thành công**: Cả hai đều cao, Adam nhỉnh hơn một chút (96.80% vs 96.40%).
- **Reward**:
  - Adam đạt **reward cuối và reward trung bình ở Episode 1000** cao hơn rõ rệt → học nhanh hơn, hội tụ sớm.
- **Tiêu thụ nhiên liệu**: RMSprop tiết kiệm hơn, nhưng sự chênh lệch không lớn.

**Kết luận**
Adam có lợi thế ở tốc độ học và khả năng hội tụ nhanh nhờ điều chỉnh tốc độ học theo tham số. Có khả năng phát triển thêm trong tương lai


#### Adam với learning rate khác

In [ ]:
# trainer = DQNTrain(
#     env=env,
#     optimizer='adam',
#     epsilon=1.0,
#     min_epsilon=0.01,
#     decay=0.995,
#     gamma=0.99,
#     batch_size=128,
#     episodes=2000,
#     target_update_freq=10,
#     memory_size=100000,
#     learning_rate=1e-3
# )

In [ ]:
# rewards, losses, fuel_usage = trainer.train()

Training DQN: 100%|██████████| 2000/2000 [32:28<00:00,  1.03it/s, Reward=264, Epsilon=0.01, Loss=1.76e+4, Fuel Usage=12.2]  
- Episode 0, Average Reward: nan
- Episode 100, Average Reward: -136.95
- Episode 200, Average Reward: -80.68
- Episode 300, Average Reward: -110.11
- Episode 400, Average Reward: -68.13
- Episode 500, Average Reward: 19.42
- Episode 600, Average Reward: -150.49
- Episode 700, Average Reward: -192.99
- Episode 800, Average Reward: -55.63
- Episode 900, Average Reward: -1.02
- Episode 1000, Average Reward: 30.97
- Episode 1100, Average Reward: 48.55
- Episode 1200, Average Reward: -50.40
- Episode 1300, Average Reward: 14.16
- Episode 1400, Average Reward: 115.29
- Episode 1500, Average Reward: 134.05
- Episode 1600, Average Reward: 93.36
- Episode 1700, Average Reward: -6.29 
- Episode 1800, Average Reward: -190.80 
- Episode 1900, Average Reward: -71.62

In [ ]:
# trainer.plot_results()

<div style="display: flex; flex-direction: column; height: 600px;">

  <!-- Hàng trên: 2 ảnh cạnh nhau -->
  <div style="flex: 1; display: flex;">
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/adam-lr-e3.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh trái"/>
    </div>
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/adam-lr-e31.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh phải"/>
    </div>
  </div>

  <!-- Hàng dưới: 1 ảnh chiếm toàn bộ chiều ngang -->
  <div style="flex: 1; overflow: hidden;">
    <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/adam-lr-e32.png?raw=true" 
         style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh dưới"/>
  </div>

</div>


In [ ]:
# trainer.save_model("dqn_lunarlander-2k-adam-1e-3.pth")

In [ ]:
# trainer.load_model("dqn_lunarlander-2k-adam-1e-3.pth")

In [ ]:
# trainer.test_model(episodes=500)

Success: 187/500 | Success Rate: 37.40%

In [ ]:
# trainer.display_sample_video(sample_video=5)

<video autoplay controls src="...">
  The “video” tag is not supported by your browser.
</video>

#### **So sánh**

| Tiêu chí                        | Adam (LR = 1e-4)     | Adam (LR = 1e-3)     |
|--------------------------------|----------------------|----------------------|
| Tổng số Epoch                  | 2000                 | 2000                 |
| Thời gian huấn luyện           | 34 phút 30 giây      | 32 phút 28 giây      |
| Tốc độ huấn luyện              | ~1.04 giây/iter      | ~1.03 giây/iter      |
| Reward cuối cùng               | 276                  | 264                  |
| Loss cuối cùng                 | 1.87e+3              | 1.76e+4              |
| Fuel Usage cuối                | 14.4                 | 12.2                 |
| Tỉ lệ thành công               | 96.80% (484/500)     | 37.40% (187/500)     |
| Reward TB @ Episode 1000       | 260.68               | 30.97                |
| Reward TB @ Episode 1900       | 248.64               | -71.62               |

**Nhận xét**

- **Thời gian & tốc độ huấn luyện**: LR = 1e-3 nhanh hơn nhẹ, nhưng không đáng kể.
- **Tỉ lệ thành công**: Giảm mạnh từ 96.8% xuống 37.4% khi tăng LR → mô hình học kém hiệu quả.
- **Reward**:
  - LR = 1e-4 cho reward cao, ổn định.
  - LR = 1e-3 cho reward biến động lớn, thậm chí âm ở cuối quá trình.
- **Loss**: Với LR cao, loss tăng vọt gần 10 lần → dấu hiệu mô hình không hội tụ.
- **Fuel Usage**: Ít hơn với LR = 1e-3, nhưng đi kèm hiệu suất thấp → không phải là lợi thế.

**Kết luận**

Tốc độ học **1e-3 là quá cao**, dẫn đến mô hình không ổn định và kém hiệu quả. Adam với **LR = 1e-4** cho kết quả **tốt và đáng tin cậy hơn**. Việc chọn LR phù hợp là **cực kỳ quan trọng trong huấn luyện DQN**, tránh hiện tượng mô hình phân kỳ hoặc không học được hành vi mong muốn.


#### Adam với discount factor khác

In [ ]:
# trainer = DQNTrain(
#     env=env,
#     optimizer='adam',
#     epsilon=1.0,
#     min_epsilon=0.01,
#     decay=0.995,
#     gamma=0.9,
#     batch_size=128,
#     episodes=2000,
#     target_update_freq=10,
#     memory_size=100000,
#     learning_rate=1e-4
# )

In [ ]:
# rewards, losses, fuel_usage = trainer.train()

Training DQN: 100%|██████████| 2000/2000 [1:19:27<00:00,  2.38s/it, Reward=-82, Epsilon=0.01, Loss=4.08e+3, Fuel Usage=53.2]  
-    Episode 0, Average Reward: nan
-    Episode 100, Average Reward: -155.28
-    Episode 200, Average Reward: -68.23
-    Episode 300, Average Reward: -53.13
-    Episode 400, Average Reward: -66.99
-    Episode 500, Average Reward: -86.92
-    Episode 600, Average Reward: -85.75
-    Episode 700, Average Reward: -86.07
-    Episode 800, Average Reward: -90.67
-    Episode 900, Average Reward: -98.86
-    Episode 1000, Average Reward: -107.91
-    Episode 1100, Average Reward: -113.87
-    Episode 1200, Average Reward: -110.60
-    Episode 1300, Average Reward: -120.69
-    Episode 1400, Average Reward: -115.58
-    Episode 1500, Average Reward: -106.18
-    Episode 1600, Average Reward: -110.01
-    Episode 1700, Average Reward: -107.90
-    Episode 1800, Average Reward: -115.30
-    Episode 1900, Average Reward: -117.88

In [ ]:
# trainer.plot_results()

<div style="display: flex; flex-direction: column; height: 600px;">

  <!-- Hàng trên: 2 ảnh cạnh nhau -->
  <div style="flex: 1; display: flex;">
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/dqn-gamma.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh trái"/>
    </div>
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/dqn-gamma1.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh phải"/>
    </div>
  </div>

  <!-- Hàng dưới: 1 ảnh chiếm toàn bộ chiều ngang -->
  <div style="flex: 1; overflow: hidden;">
    <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/dqn-gamma2.png?raw=true" 
         style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh dưới"/>
  </div>

</div>

In [ ]:
# trainer.save_model("dqn_lunarlander-2k-adam-gamma-09.pth")

In [ ]:
# trainer.load_model("dqn_lunarlander-2k-adam-gamma-09.pth")

In [ ]:
# trainer.test_model(episodes=500)

100%|██████████| 500/500 [00:48<00:00, 10.36it/s]
Success: 0/500 | Success Rate: 0.00%

In [ ]:
# trainer.display_sample_video(sample_video=5)

<video autoplay controls src="...">
  The “video” tag is not supported by your browser.
</video>

#### **So sánh**

**Bảng tổng hợp kết quả**

| Tiêu chí                          | Adam (γ=0.99)        | Adam (γ=0.9)           |
|----------------------------------|----------------------|------------------------|
| Tổng số Episodes                 | 2000                 | 2000                   |
| Thời gian huấn luyện             | 34 phút 30 giây      | 1 giờ 19 phút 27 giây  |
| Tốc độ huấn luyện                | ~1.04 giây/iteration | ~2.38 giây/iteration   |
| Reward cuối cùng                 | 276                  | -82                    |
| Loss cuối cùng                   | 1.87e+3              | 4.08e+3                |
| Fuel Usage cuối cùng             | 14.4                 | 53.2                   |
| Tỉ lệ thành công                 | **96.80% (484/500)** | **0.00% (0/500)**      |
| Reward trung bình Episode 1000   | 260.68               | -107.91                |
| Reward trung bình Episode 1900   | 248.64               | -117.88                |


**Phân tích kết quả**

- **Thời gian và tốc độ huấn luyện**:  
  Với γ = 0.9, thời gian huấn luyện kéo dài hơn gấp đôi so với γ = 0.99. Đồng thời, tốc độ xử lý mỗi iteration cũng chậm hơn đáng kể.

- **Tỉ lệ thành công**:  
  Là yếu tố quyết định trong bài toán. Khi γ = 0.99, tác nhân đạt tỉ lệ thành công 96.80%, trong khi γ = 0.9 hoàn toàn không thành công trong bất kỳ lần thử nghiệm nào.

- **Giá trị Reward và Loss**:  
  Với γ = 0.99, mô hình đạt được reward cuối cùng cao và reward trung bình dương ổn định. Trong khi đó, γ = 0.9 dẫn đến reward âm và không có dấu hiệu cải thiện về sau. Đồng thời, loss cuối cùng của γ = 0.9 cao hơn nhiều, cho thấy quá trình học gặp khó khăn trong việc tối ưu hàm mất mát.

- **Mức tiêu thụ nhiên liệu (Fuel Usage)**:  
  Với γ = 0.9, lượng nhiên liệu tiêu thụ cao gấp nhiều lần, phản ánh một chiến lược không hiệu quả và lãng phí tài nguyên do không học được cách hạ cánh tối ưu.

**Vai trò của hệ số chiết khấu gamma (γ)**

Gamma là hệ số chiết khấu trong học tăng cường, phản ánh mức độ tác nhân đánh giá phần thưởng tương lai so với phần thưởng hiện tại.

- **γ gần 1 (ví dụ: 0.99)**:  
  Tác nhân đánh giá cao phần thưởng trong tương lai. Điều này phù hợp với các bài toán có chuỗi hành động dài như Lunar Lander, nơi hành động hiện tại chỉ góp phần vào phần thưởng sau cùng.

- **γ nhỏ hơn (ví dụ: 0.9)**:  
  Tác nhân thiên về phần thưởng ngắn hạn. Trong bối cảnh bài toán Lunar Lander, điều này khiến mô hình không học được chiến lược dài hạn, bỏ qua các phần thưởng quan trọng ở cuối quá trình.

**Kết luận**

Giá trị gamma (γ) có ảnh hưởng quyết định đến hiệu quả học của tác nhân trong các bài toán tăng cường mang tính dài hạn. Trong bài toán Lunar Lander, γ = 0.99 mang lại hiệu suất vượt trội: tốc độ huấn luyện nhanh, reward cao, loss thấp và tỉ lệ thành công gần như tuyệt đối. Ngược lại, γ = 0.9 dẫn đến thất bại hoàn toàn. Kết quả này nhấn mạnh tầm quan trọng của việc lựa chọn tham số γ phù hợp với cấu trúc phần thưởng và chiều dài chuỗi hành động trong từng bài toán cụ thể.


### e. Kết luận

Thuật toán **DQN** coi như đã giải được bài toán **Lunar Lander**, với hiệu suất có thể đạt đến **~99%** sau đủ số episode, như minh chứng bởi mô hình `lunarlander-5k.pth`.

**Lý do hiệu suất cao:**

- **Không gian hành động nhỏ:** Chỉ 4 hành động rời rạc → dễ tối ưu Q-value cho từng hành động.
- **Trạng thái rõ ràng:** 8 giá trị số thực mô tả đầy đủ vị trí, tốc độ, góc và tiếp xúc chân → mạng dễ học.
- **Phần thưởng có cấu trúc:** Thưởng/phạt rõ ràng thúc đẩy hành vi hạ cánh đúng cách.
- **Môi trường ổn định:** Ít yếu tố ngẫu nhiên (gió, nhiễu loạn) → quá trình học ổn định hơn.
- **Kỹ thuật ổn định hóa:**
  - **Replay Buffer:** Giảm tương quan dữ liệu.
  - **Target Network:** Tránh thay đổi Q-target quá nhanh → ổn định gradient.


DQN rất phù hợp cho các môi trường **trạng thái liên tục nhưng đơn giản**, **hành động rời rạc**, và **phản hồi có cấu trúc**, như Lunar Lander.


In [ ]:
# Chứng minh hiệu suất của mô hình đã được huấn luyện 5k episodes
# test = DQNTrain(
#     env=env,
#     optimizer='adam',
#     epsilon=1.0,
#     min_epsilon=0.01,
#     decay=0.995,
#     gamma=0.99,
#     batch_size=128,
#     episodes=2000,
#     target_update_freq=10,
#     memory_size=100000,
#     learning_rate=1e-4
# )

# test.load_model("lunarlander-5k.pth")
# test.test_model(episodes=500)
# test.display_sample_video(sample_video=1)

100%|██████████| 500/500 [00:17<00:00, 29.25it/s]
Success: 494/500 | Success Rate: 98.80%

<video autoplay controls src="...">
  The “video” tag is not supported by your browser.
</video>

# 5. Double Deep Q-Network
---

Với mô hình LunarLander truyền thống, khi không có sự tác động mạnh của môi trường như gió hoặc nhiễu loạn gió, đặc trưng học nhanh của mô hình (đặc biệt khi sử dụng thuật toán tối ưu `Adam`) giúp `DQN` đạt hiệu suất rất cao. Điều này có thể là do **Không gian trạng thái ổn định và dễ dự đoán**

Khi loại bỏ các yếu tố gây nhiễu như gió, môi trường trở nên nhất quán hơn giữa các lần huấn luyện. Điều này giúp mạng neuron trong DQN dễ dàng học được các quy luật điều khiển phù hợp, mà không cần phải xử lý các tình huống ngoại lệ hay biến động bất thường.

## a. Môi trường gió

Chúng ta sẽ test thử mô hình DQN với môi trường nhiễu động do gió:
- wind_power=15 (tốc độ gió)
- turbulence_power=1.5 (nhiễu loạn gió)

In [ ]:
win_env = gym.make("LunarLander-v3",gravity=-10.0, enable_wind=True, wind_power=15.0, turbulence_power=1.5)

**Vì môi trường gió khá phức tạp, nên ta sẽ tăng episodes lên 2500, epsilon_decay lên 0.997 giúp agent học được nhiều hơn, buffer replay lên 200k để mô hình có nhiều mẫu để học hơn.**

## b. DQN Training

Ta sẽ kế thừa các function từ class `DQNTrain`, override phương thức `test_model` và `display_sample_video` với class `DQNTrainWind` để test và xuất video dưới môi trường có gió.

In [ ]:
class DQNTrainWind(DQNTrain):
    def __init__(self, env, optimizer, epsilon, min_epsilon, decay, gamma, batch_size, episodes, target_update_freq, memory_size, learning_rate):
        super().__init__(env, optimizer, epsilon, min_epsilon, decay, gamma, batch_size, episodes, target_update_freq, memory_size, learning_rate)
    
    def test_model(self, episodes=500):
        total_success = 0
        # Sử dụng môi trường có gió và trọng lực khác
        testing = gym.make("LunarLander-v3",gravity=-10.0, enable_wind=True, wind_power=15.0, turbulence_power=1.5)
        
        for episode in tqdm(range(episodes)):
            state, _ = testing.reset()
            done = False
            total_reward = 0
            
            while not done:
                action = self.get_target_action(state)
                state, reward, terminated, truncated, _ = testing.step(action)
                total_reward += reward
                done = terminated or truncated
            if total_reward >= 200:
                total_success += 1
        print(f"Success: {total_success}/{episodes} | Success Rate: {total_success / episodes * 100:.2f}%")
        testing.close()
        
        
    def display_sample_video(self, sample_video=1):
        # Hiển thị video mẫu của mô hình đã được huấn luyện trong môi trường có gió và trọng lực khác
        video_render = gym.make("LunarLander-v3",gravity=-10.0, enable_wind=True, wind_power=15.0, turbulence_power=1.5, render_mode="rgb_array")
        video_render = RecordVideo(video_render, "win-videos", episode_trigger=lambda x: True)
        
        for _ in tqdm(range(sample_video)):
            state, _ = video_render.reset()
            done = False
            while not done:
                action = self.get_target_action(state)
                state, reward, terminated, truncated, _ = video_render.step(action)
                done = terminated or truncated
            video_render.close()
        video_files = glob.glob("win-videos/*.mp4")
        return Video(video_files[-1])
        

In [ ]:
# win_env_dqn_train = DQNTrainWind(
#     env=win_env,
#     optimizer='adam',
#     epsilon=1.0,
#     min_epsilon=0.01,
#     decay=0.997,
#     gamma=0.99,
#     batch_size=128,
#     episodes=2500,
#     target_update_freq=10,
#     memory_size=200000,
#     learning_rate=1e-4
# )

In [ ]:
# reward, losses, fuel_usage = win_env_dqn_train.train()

Training DQN: 100%|██████████| 2500/2500 [1:09:56<00:00,  1.68s/it, Reward=252, Epsilon=0.01, Loss=3.01e+3, Fuel Usage=22]    
- Episode 0, Average Reward: nan
- Episode 100, Average Reward: -262.69
- Episode 200, Average Reward: -151.65
- Episode 300, Average Reward: -101.52
- Episode 400, Average Reward: -85.88
- Episode 500, Average Reward: -96.02
- Episode 600, Average Reward: -122.30
- Episode 700, Average Reward: -140.99
- Episode 800, Average Reward: -103.52
- Episode 900, Average Reward: -98.36
- Episode 1000, Average Reward: -73.21
- Episode 1100, Average Reward: -85.10
- Episode 1200, Average Reward: -76.41
- Episode 1300, Average Reward: -76.36
- Episode 1400, Average Reward: 50.29
- Episode 1500, Average Reward: 160.69
- Episode 1600, Average Reward: 122.38
- Episode 1700, Average Reward: 116.17
- Episode 1800, Average Reward: 203.61
- Episode 1900, Average Reward: 192.99
- Episode 2000, Average Reward: 213.29
- Episode 2100, Average Reward: 216.32
- Episode 2200, Average Reward: 224.50
- Episode 2300, Average Reward: 211.06
- Episode 2400, Average Reward: 207.99


In [ ]:
# win_env_dqn_train.plot_results()

<div style="display: flex; flex-direction: column; height: 600px;">

  <!-- Hàng trên: 2 ảnh cạnh nhau -->
  <div style="flex: 1; display: flex;">
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/dqn-win.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh trái"/>
    </div>
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/dqn-win1.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh phải"/>
    </div>
  </div>

  <!-- Hàng dưới: 1 ảnh chiếm toàn bộ chiều ngang -->
  <div style="flex: 1; overflow: hidden;">
    <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/dqn-win2.png?raw=true" 
         style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh dưới"/>
  </div>

</div>


In [ ]:
# win_env_dqn_train.save_model("dqn_lunarlander-win-env-2k5-adam.pth")

In [ ]:
# win_env_dqn_train.load_model("dqn_lunarlander-win-env-2k5-adam.pth")

In [ ]:
# win_env_dqn_train.test_model(episodes=500)

100%|██████████| 500/500 [00:29<00:00, 16.72it/s]
Success: 393/500 | Success Rate: 78.60%

In [ ]:
# win_env_dqn_train.display_sample_video(sample_video=5)

<video autoplay controls src="...">
  The “video” tag is not supported by your browser.
</video>

**Nhận xét:**
Có thể thấy rằng khi hoạt động trong môi trường có sự nhiễu loạn (như gió và biến động môi trường), hiệu suất của DQN giảm rõ rệt. Cụ thể, mức average reward > 200 chỉ đạt được sau khoảng 1800 episodes, so với mốc 800 episodes trong môi trường tiêu chuẩn. Tỷ lệ thành công (success rate) cũng giảm, chỉ còn 78%, thay vì >90% như trước. Đồng thời, thời gian huấn luyện tăng đáng kể lên đến khoảng 1h05 phút (2500ep) — cho thấy mức độ khó khăn tăng lên khi môi trường trở nên bất định hơn.
Điều này có thể là do:
- DQN dễ overfit vào các mẫu trạng thái quen thuộc: DQN có xu hướng học "quá mức" vào các mẫu quen thuộc, không có khả năng thay đổi để ứng phó với môi trường mới.

## c. Tối ưu với DDQN

Nhận thấy sự thiếu sót của DQN trong môi trường có nhiễu loạn, dưới đây là triển khai của mô hình Double DQN (DDQN) cho hiệu suất tốt hơn ở môi trường nhiễu loạn.

Sự nâng cấp chính của DDQN so với DQN là cách tính target-Q trong bước huấn luyện

$\implies$ DDQN giảm hiện tượng quá ước lượng (overestimation bias) của Q-value

Đây là code trong DQN:
```bash
next_q_values = target_net(next_states).max(1)[0].unsqueeze(1)
targets = rewards + gamma * next_q_values * (1 - dones)
```
Ở đây:
- target_net(next_states).max(1)[0] chọn hành động tốt nhất theo giá trị Q tối đa trong target_net.
- Tuy nhiên, hành động tốt nhất cũng được chọn bởi chính target_net, dẫn đến overestimation (ước lượng Q-value quá cao).

Đây là thay đổi chính trong DDQN:
```bash
next_actions = self.q_net(next_states).argmax(1).unsqueeze(1)
next_q_values = self.target_net(next_states).gather(1, next_actions)
targets = rewards + self.gamma * next_q_values * (1 - dones)
```

Ý tưởng:
- Chọn hành động tốt nhất từ q_net: next_actions = self.q_net(next_states).argmax(1)
- Đánh giá giá trị của hành động đó bằng target_net: self.target_net(next_states).gather(1, next_actions)

Tách biệt vai trò:
- `q_net`: dùng để chọn hành động tốt nhất.
- `target_net`: dùng để đánh giá hành động đó.

$\implies$ Điều này giúp giảm overestimation và tăng tính ổn định khi huấn luyện.


In [ ]:
class DDQNTrainWind(DQNTrainWind):
    def __init__(self, env, optimizer, epsilon, min_epsilon, decay, gamma, batch_size, episodes, target_update_freq, memory_size, learning_rate):
        super().__init__(env, optimizer, epsilon, min_epsilon, decay, gamma, batch_size, episodes, target_update_freq, memory_size, learning_rate)
    
    def train_step(self):
        '''
        - Các phần tương tự như trong hàm train_step của DQN
        - Tuy nhiên, trong hàm này, chúng ta sẽ sử dụng mạng Q mục tiêu để tính toán giá trị Q mục tiêu
        - Chúng ta sẽ sử dụng mạng Q chính để tính toán giá trị Q hiện tại
        - Điều này giúp giảm thiểu bias trong việc ước lượng giá trị Q
        '''
        if len(self.memory) < self.batch_size:
            return 0
        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        states = torch.FloatTensor(np.array(states)).to(device)
        actions = torch.LongTensor(actions).unsqueeze(1).to(device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(device)
        next_states = torch.FloatTensor(np.array(next_states)).to(device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(device)

        q_values = self.q_net(states).gather(1, actions)

        with torch.no_grad():
            next_actions = self.q_net(next_states).argmax(1).unsqueeze(1)
            next_q_values = self.target_net(next_states).gather(1, next_actions)
            targets = rewards + self.gamma * next_q_values * (1 - dones)

        loss = nn.MSELoss()(q_values, targets)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

In [ ]:
ddqn_trainer = DDQNTrainWind(
    env=win_env,
    optimizer='adam',
    epsilon=1.0,
    min_epsilon=0.01,
    decay=0.997,
    gamma=0.99,
    batch_size=128,
    episodes=2500,
    target_update_freq=10,
    memory_size=200000,
    learning_rate=1e-4
)

In [ ]:
# rewards, losses, fuel_usage = ddqn_trainer.train()

Training DDQNWind: 100%|██████████| 2500/2500 [54:24<00:00,  1.31s/it, Reward=289, Epsilon=0.01, Loss=3.9e+3, Fuel Usage=22.3]   
-    Episode 0, Average Reward: nan
-    Episode 100, Average Reward: -233.83
-    Episode 200, Average Reward: -154.99
-    Episode 300, Average Reward: -104.12
-    Episode 400, Average Reward: -118.83
-    Episode 500, Average Reward: -110.05
-    Episode 600, Average Reward: -120.17
-    Episode 700, Average Reward: -118.32
-    Episode 800, Average Reward: -48.29
-    Episode 900, Average Reward: 115.21
-    Episode 1000, Average Reward: 163.66
-    Episode 1100, Average Reward: 121.09
-    Episode 1200, Average Reward: 95.76
-    Episode 1300, Average Reward: 57.27
-    Episode 1400, Average Reward: 25.05
-    Episode 1500, Average Reward: 27.55
-    Episode 1600, Average Reward: 88.26
-    Episode 1700, Average Reward: 93.46
-    Episode 1800, Average Reward: 124.60
-    Episode 1900, Average Reward: 185.44
-    Episode 2000, Average Reward: 198.50
-    Episode 2100, Average Reward: 199.99
-    Episode 2200, Average Reward: 202.52
-    Episode 2300, Average Reward: 221.71
-    Episode 2400, Average Reward: 242.54

In [ ]:
# ddqn_trainer.plot_results()

<div style="display: flex; flex-direction: column; height: 600px;">

  <!-- Hàng trên: 2 ảnh cạnh nhau -->
  <div style="flex: 1; display: flex;">
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/ddqn-win.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh trái"/>
    </div>
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/ddqn-win1.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh phải"/>
    </div>
  </div>

  <!-- Hàng dưới: 1 ảnh chiếm toàn bộ chiều ngang -->
  <div style="flex: 1; overflow: hidden;">
    <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/ddqn-win2.png?raw=true" 
         style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh dưới"/>
  </div>

</div>


In [ ]:
# ddqn_trainer.save_model("ddqn_lunarlander-win-env-2k5-adam.pth")

In [ ]:
# ddqn_trainer.load_model("ddqn_lunarlander-win-env-2k5-adam.pth")

In [ ]:
# ddqn_trainer.test_model(episodes=500)

100%|██████████| 500/500 [00:28<00:00, 17.68it/s]
Success: 430/500 | Success Rate: 86.00%

In [ ]:
# ddqn_trainer.display_sample_video(sample_video=5)

<video autoplay controls src="...">
  The “video” tag is not supported by your browser.
</video>

Có thể thấy DDQN có tốc độ training nhỉnh hơn so với DQN (54p và 1h10p) trong khi độ chính xác cao hơn
- Đạt success rate 86% cao hơn với success rate 78% từ DQN

Lý do: 
- Giảm overestimation nhờ tách biệt quá trình chọn và đánh giá hành động.
- Hội tụ ổn định hơn trong môi trường nhiễu.
- Chính sách học được tổng quát tốt hơn, giúp tăng độ chính xác và hiệu suất.

**Tuy nhiên: Hiệu suất đạt được khi môi trường có gió vẫn chưa lý tưởng (86%)**

Nguyên nhân:
- Độ phức tạp của môi trường có gió rất cao, gió thổi mạnh ngoài ra còn có sự nhiễu của gió nên ảnh hưởng đến quyết định của agent. Ví dụ như episode(n), ở step(n), ở môi trường bình thường chọn action(n) sẽ cho ta đạt kết quả tốt, từ đó agent cứ tiếp tục xử lí như vậy. Nhưng ở episode(n+1), vẫn ở step đó nhưng mỗi lần sẽ cho ra một kết quả khác nhau.
- Agent có thể nghĩ rằng một hành động tốt lại dẫn đến kết quả xấu do tác động của gió, và ngược lại.

# 6. Cải tiến DDQN
---

### a. Nhân xét

Chúng ta vừa thử huấn luyện agent trong môi trường có gió với các thuật toán như `DQN` và `DDQN`. 

Tuy nhiên hiệu quả đem lại của các thuật toán chưa được tốt, việc học vẫn còn chưa ổn định. Chúng ta sẽ thiết kế lại một mạng neuron mới để cải thiện thuật toán `DDQN`.

### b. Triển khai Code 

Trước tiên, ta cần khởi tạo lại một neural network `DDQNWindNetwork` với đầu vào bao gồm
- state_dim: Kích thước của trạng thái hiện tại
- prev_state_dim: Kích thước của trạng thái trước đó
- action_dim: Số lượng hành động có thể chọn

Mạng sẽ nhận vào:
- Trạng thái hiện tại $(step(n))$
- Trạng thái trước đó $(step(n-1))$
- Hành động trước đó $(action(n-1))$

=> Output: Dự đoán hành động hiện tại $(action(n))$

In [ ]:
class DDQNWindNetwork(nn.Module):
    def __init__(self, state_dim, prev_state_dim, action_dim):
        super(DDQNWindNetwork, self).__init__()
        
        # Kích thước đầu vào = state_dim + prev_state_dim + 1 (hành động trước đó)
        input_dim = state_dim + prev_state_dim + 1
        self.fc1 = nn.Linear(input_dim, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, action_dim)
        self.relu = nn.ReLU()
        
    def forward(self, state, prev_state, prev_action):
        combined = torch.cat([state, prev_state, prev_action], dim=1)
        x = self.relu(self.fc1(combined))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

Ta sẽ định nghĩa lại class `DDQNOptim` (kế thừa từ `DDQNTrainWind` những hàm chung), override lại những hàm như `__init__`, `select_action`, `train_step`, `train`, `get_target_action` với mục đích truyền vào trạng thái hiện tại + trạng thái cũ + action của trạng thái cũ từ đó đưa ra hành động hợp lý hiện tại.

In [ ]:
class DDQNOptim(DDQNTrainWind):
    def __init__(self, env, optimizer, epsilon, min_epsilon, decay, 
                 gamma, batch_size, episodes, target_update_freq, 
                 memory_size, learning_rate):

        self.env = env
        self.episodes = episodes
        self.epsilon = epsilon
        self.min_epsilon = min_epsilon
        self.decay = decay
        self.gamma = gamma
        self.batch_size = batch_size
        self.target_update_freq = target_update_freq
        self.memory = deque(maxlen=memory_size)
        self.fuel_usage = []
        self.rewards = []
        self.losses = []

        state_dim = env.observation_space.shape[0]
        action_dim = env.action_space.n
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        #định nghĩa mạng Q và mạng Q mục tiêu với kiến trúc mới
        self.q_net = DDQNWindNetwork(state_dim, state_dim, action_dim).to(device)
        self.target_net = DDQNWindNetwork(state_dim, state_dim, action_dim).to(device)
        self.target_net.load_state_dict(self.q_net.state_dict())
        self.target_net.eval()
        
        if optimizer == 'adam':
            self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=learning_rate)
        elif optimizer == 'rmsprop':
            self.optimizer = torch.optim.RMSprop(self.q_net.parameters(), lr=learning_rate)
        elif optimizer == 'sgd':
            self.optimizer = torch.optim.SGD(self.q_net.parameters(), lr=learning_rate)
        
        # Khởi tạo trạng thái và hành động trước đó
        self.prev_state = None
        self.prev_action = None
        
    
    def select_action(self, state):
        # nếu không có gì thì khởi tạo =0 hết
        if self.prev_state is None:
            self.prev_state = np.zeros_like(state)
            self.prev_action = np.zeros(1)
        
        # Chọn hành động ngẫu nhiên với xác suất epsilon
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, self.env.action_space.n)
        else:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
                prev_state_tensor = torch.FloatTensor(self.prev_state).unsqueeze(0).to(self.device)
                prev_action_tensor = torch.FloatTensor([self.prev_action]).unsqueeze(0).to(self.device)
                
                # Sử dụng mạng Q để dự đoán hành động
                return self.q_net(state_tensor, prev_state_tensor, prev_action_tensor).argmax().item()
    
    def train_step(self):
        if len(self.memory) < self.batch_size:
            return 0
        
        # Lấy mẫu ngẫu nhiên từ bộ nhớ
        batch = random.sample(self.memory, self.batch_size)
        states, prev_states, prev_actions, actions, rewards, next_states, next_prev_states, next_prev_actions, dones = zip(*batch)
        
        states = np.array(states)
        prev_states = np.array(prev_states)
        prev_actions = np.array(prev_actions)
        actions = np.array(actions)
        rewards = np.array(rewards)
        next_states = np.array(next_states)
        next_prev_states = np.array(next_prev_states)
        next_prev_actions = np.array(next_prev_actions)
        dones = np.array(dones)
        
        states = torch.FloatTensor(states).to(self.device)
        prev_states = torch.FloatTensor(prev_states).to(self.device)
        prev_actions = torch.FloatTensor(prev_actions).unsqueeze(1).to(self.device)
        actions = torch.LongTensor(actions).unsqueeze(1).to(self.device)
        rewards = torch.FloatTensor(rewards).unsqueeze(1).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        next_prev_states = torch.FloatTensor(next_prev_states).to(self.device)
        next_prev_actions = torch.FloatTensor(next_prev_actions).unsqueeze(1).to(self.device)
        dones = torch.FloatTensor(dones).unsqueeze(1).to(self.device)
        
        # Tính giá trị Q hiện tại
        q_values = self.q_net(states, prev_states, prev_actions).gather(1, actions)
        
        # Tính giá trị Q mục tiêu
        with torch.no_grad():
            # Tìm hành động tốt nhất cho trạng thái tiếp theo
            next_q_values = self.q_net(next_states, next_prev_states, next_prev_actions)
            best_actions = next_q_values.argmax(dim=1, keepdim=True)
            
            # Tính giá trị Q mục tiêu sử dụng mạng target
            next_target_q_values = self.target_net(next_states, next_prev_states, next_prev_actions)
            max_next_q = next_target_q_values.gather(1, best_actions)
            
            # Tính Q mục tiêu
            targets = rewards + self.gamma * max_next_q * (1 - dones)
        
        loss = nn.MSELoss()(q_values, targets)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss.item()
    
    def train(self):
        pbar = tqdm(range(self.episodes), desc="Training DDQN Wind Optimized")
        
        for ep in pbar:
            state, _ = self.env.reset()
            self.prev_state = np.zeros_like(state)  # Trạng thái trước đó = 0
            self.prev_action = 0  # Hành động trước đó = 0
            
            total_reward = 0
            total_loss = 0
            total_fuel = 0
            done = False
            
            while not done:
                # Chọn hành động
                action = self.select_action(state)
                
                # Thực hiện hành động
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                
                self.memory.append((
                    state, self.prev_state, self.prev_action, action, reward, 
                    next_state, state, action, float(done)
                ))
                
                # Huấn luyện
                loss = self.train_step()
                total_loss += loss
                total_reward += reward
                total_fuel += FUEL_COST[action]
                
                # Cập nhật trạng thái và hành động trước đó
                self.prev_state = state
                self.prev_action = action
                state = next_state
            
            # Cập nhật mạng Q mục tiêu
            if ep % self.target_update_freq == 0:
                self.target_net.load_state_dict(self.q_net.state_dict())
            
            if ep % 100 == 0:
                average_reward = np.mean(self.rewards[-100:]) if len(self.rewards) > 0 else 0
                print(f"Episode {ep}, Average Reward: {average_reward:.2f}")
            
            self.epsilon = max(self.min_epsilon, self.epsilon * self.decay)
            
            self.rewards.append(total_reward)
            self.losses.append(total_loss)
            self.fuel_usage.append(total_fuel)
            
            pbar.set_postfix({
                "Reward": total_reward,
                "Epsilon": self.epsilon,
                "Loss": total_loss,
                "Fuel Usage": total_fuel
            })
        
        return self.rewards, self.losses, self.fuel_usage
    
    def get_target_action(self, state):
        '''
        Chọn hành động tốt nhất từ mạng Q để kiểm tra (không có epsilon-greedy)
        '''
        # Nếu không có trạng thái trước đó, tạo một trạng thái 0
        if self.prev_state is None:
            self.prev_state = np.zeros_like(state)
            self.prev_action = np.zeros(1)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            prev_state_tensor = torch.FloatTensor(self.prev_state).unsqueeze(0).to(self.device)
            prev_action_tensor = torch.FloatTensor([self.prev_action]).unsqueeze(0).to(self.device)
            return self.q_net(state_tensor, prev_state_tensor, prev_action_tensor).argmax().item()
        
    
    def test_model(self, episodes=500):
        total_success = 0
        testing = gym.make("LunarLander-v3",gravity=-10.0, enable_wind=True, wind_power=15.0, turbulence_power=1.5)

        for episode in tqdm(range(episodes)):
            state, _ = testing.reset()
            self.prev_state = np.zeros_like(state) 
            self.prev_action = 0

            done = False
            total_reward = 0

            while not done:
                action = self.get_target_action(state)
                next_state, reward, terminated, truncated, _ = testing.step(action)
                done = terminated or truncated

                total_reward += reward
                self.prev_state = state
                self.prev_action = action
                state = next_state

            if total_reward >= 200:
                total_success += 1

        print(f"Success: {total_success}/{episodes} | Success Rate: {total_success / episodes * 100:.2f}%")
        testing.close()

    def display_sample_video(self, sample_video=5):
        video_render = gym.make("LunarLander-v3",gravity=-10.0, enable_wind=True, wind_power=15.0, turbulence_power=1.5, render_mode="rgb_array")
        video_render = RecordVideo(video_render, "videos", episode_trigger=lambda x: True)

        for _ in tqdm(range(sample_video)):
            state, _ = video_render.reset()
            self.prev_state = np.zeros_like(state)
            self.prev_action = 0 

            done = False

            while not done:
                action = self.get_target_action(state)
                next_state, reward, terminated, truncated, _ = video_render.step(action)
                done = terminated or truncated
                self.prev_state = state
                self.prev_action = action
                state = next_state

            video_render.close()

        video_files = glob.glob("videos/*.mp4")
        return Video(video_files[-1])

### c. Kết quả

In [ ]:
# ddqn_optim = DDQNOptim(
#     env=win_env,
#     optimizer='adam',
#     epsilon=1.0,
#     min_epsilon=0.01,
#     decay=0.997,
#     gamma=0.99,
#     batch_size=128,  
#     episodes=2500,
#     target_update_freq=10,
#     memory_size=200000,
#     learning_rate=1e-4
# )

In [ ]:
# rewards, losses, fuel_usage = ddqn_optim.train()

Training DDQN Wind Optimized: 100%|██████████| 2500/2500 [44:24<00:00,  1.07s/it, Reward=284, Epsilon=0.01, Loss=1.82e+3, Fuel Usage=13.9] 
-    Episode 0, Average Reward: 0.00
-    Episode 100, Average Reward: -252.31
-    Episode 200, Average Reward: -166.94
-    Episode 300, Average Reward: -89.21
-    Episode 400, Average Reward: -94.44
-    Episode 500, Average Reward: -50.89
-    Episode 600, Average Reward: -11.90
-    Episode 700, Average Reward: 54.86
-    Episode 800, Average Reward: 86.36
-    Episode 900, Average Reward: 116.05
-    Episode 1000, Average Reward: 135.79
-    Episode 1100, Average Reward: 152.44
-    Episode 1200, Average Reward: 155.93
-    Episode 1300, Average Reward: 102.30
-    Episode 1400, Average Reward: 165.24
-    Episode 1500, Average Reward: 194.16
-    Episode 1600, Average Reward: 185.43
-    Episode 1700, Average Reward: 176.14
-    Episode 1800, Average Reward: 214.19
-    Episode 1900, Average Reward: 209.00
-    Episode 2000, Average Reward: 238.84
-    Episode 2100, Average Reward: 226.54
-    Episode 2200, Average Reward: 219.77
-    Episode 2300, Average Reward: 237.90
-    Episode 2400, Average Reward: 243.15

In [ ]:
# ddqn_optim.plot_results()

<div style="display: flex; flex-direction: column; height: 600px;">

  <!-- Hàng trên: 2 ảnh cạnh nhau -->
  <div style="flex: 1; display: flex;">
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/dqn-optim.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh trái"/>
    </div>
    <div style="flex: 1; overflow: hidden;">
      <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/dqn-optim1.png?raw=true" 
           style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh phải"/>
    </div>
  </div>

  <!-- Hàng dưới: 1 ảnh chiếm toàn bộ chiều ngang -->
  <div style="flex: 1; overflow: hidden;">
    <img src="https://github.com/sleepifoxx/solve-lunarlander-v3/blob/main/images/dqn-optim2.png?raw=true" 
         style="width: 100%; height: 100%; object-fit: contain;" alt="Ảnh dưới"/>
  </div>

</div>


In [ ]:
# ddqn_optim.save_model("ddqn_optim_lunarlander-win-env-2-5k-adam.pth")

In [ ]:
# ddqn_optim.load_model("ddqn_optim_lunarlander-win-env-2-5k-adam.pth")

In [ ]:
# ddqn_optim.test_model(episodes=500)

100%|██████████| 500/500 [00:30<00:00, 16.13it/s]
Success: 457/500 | Success Rate: 91.40%

In [ ]:
# ddqn_optim.display_sample_video(sample_video=5)

<video autoplay controls src="...">
  The “video” tag is not supported by your browser.
</video>

### d. Kết luận

| Tiêu chí                        | DQN           | DDQN (Wind)    | DDQN Optimized (Wind) |
|--------------------------------|----------------|----------------|------------------------|
| Tổng số Episodes               | 2500           | 2500           | 2500                   |
| Thời gian huấn luyện           | 1h 09p 56s     | 54p 24s        | 44p 24s                |
| Tốc độ huấn luyện              | ~1.68s/iter    | ~1.31s/iter    | ~1.07s/iter            |
| Reward cuối cùng               | 252            | 289            | 284                    |
| Loss cuối cùng                 | 3.01e+3        | 3.9e+3         | 1.82e+3                |
| Fuel Usage cuối                | 22             | 22.3           | 13.9                   |
| Tỉ lệ thành công               | 78.60%         | 86.00%         | 91.40%                 |
| Reward TB @ Episode 1000       | -73.21         | 163.66         | 135.79                 |
| Reward TB @ Episode 2400       | 207.99         | 242.54         | 243.15                 |

**Nhận xét:**

- **DQN:** 
  - Huấn luyện lâu và chậm nhất.
  - Tỉ lệ thành công thấp (78.6%) và reward cải thiện chậm.
  - Reward âm đến tận ~Episode 1400.

- **DDQN (Wind):** 
  - Huấn luyện nhanh hơn DQN.
  - Reward hội tụ nhanh, đạt cao nhất (289).
  - Tỉ lệ thành công 86%, nhưng loss cao hơn do dao động trong quá trình học.

- **DDQN Optimized:** 
  - Nhanh và ổn định nhất.
  - Tỉ lệ thành công cao nhất (91.4%).
  - Loss thấp nhất → quá trình học hiệu quả hơn.
  - Tiêu thụ nhiên liệu ít nhất (13.9).

**Lý do hiệu quả của DDQN Optimized:**

- **Thêm state(n-1) + action(n-1):** 
  - Cung cấp thông tin về lịch sử, giúp mô hình hiểu rõ hơn tác động của hành động trước.
  - Tăng khả năng học hành vi có tính phụ thuộc thời gian.
  - Cải thiện tính ổn định và chất lượng chiến lược.

**Kết luận:**

DDQN Optimized là thuật toán hiệu quả nhất cho bài toán Lunar Lander trong môi trường có gió. Nó kết hợp tốc độ huấn luyện nhanh, tỉ lệ thành công cao, loss thấp và khả năng sử dụng nhiên liệu tối ưu nhờ thiết kế đầu vào giàu thông tin.


# 7. Kết luận chung
---

Dưới đây là tổng kết ngắn gọn về hiệu suất của các thuật toán học tăng cường đã được thử nghiệm trên bài toán Lunar Lander:

- **Q-learning với Q-table**: Đây là phương pháp cơ bản nhất, sử dụng bảng Q để lưu trữ các giá trị hành động-trạng thái. Tuy nhiên, mô hình này cho độ chính xác và độ ổn định thấp, đặc biệt khi không gian trạng thái lớn hoặc liên tục.

- **Deep Q-Network (DQN)**: Là một bước tiến đáng kể, DQN sử dụng mạng nơ-ron để xấp xỉ hàm Q, giúp xử lý tốt hơn các không gian trạng thái lớn. Mô hình này cho thấy độ chính xác vượt trội với bộ tham số hiệu quả, nhưng vẫn còn thiếu ổn định trong quá trình cập nhật mạng.

- **Double DQN (DDQN)**: Một phiên bản cải tiến của DQN nhằm giải quyết vấn đề ước lượng quá cao giá trị Q. DDQN mang lại sự ổn định cao hơn và hiệu suất tốt hơn so với DQN, đặc biệt trong các môi trường phức tạp có yếu tố ngẫu nhiên như gió.

- **DDQN Optimized (DDQN Optim)**: Đây là phiên bản cải tiến hiệu quả nhất, trong đó mạng nơ-ron được bổ sung thông tin từ trạng thái và hành động trước đó. Cách tiếp cận này giúp mô hình hiểu rõ hơn về động lực học của môi trường, dẫn đến độ chính xác cực kỳ cao và hiệu suất sử dụng nhiên liệu vượt trội.

Về cơ bản, `DDQN Optim` đã thành công trong việc giải quyết bài toán `Lunar Lander`, đạt được xác suất thành công trên 90% ngay cả trong môi trường khắc nghiệt với gió và nhiễu loạn.


**Bài trình bày của em xin hết, em xin chân thành cảm ơn!**

References:
- Github project: https://github.com/sleepifoxx/solve-lunarlander-v3
- Gymnasium Documentation for Lunar Lander: https://gymnasium.farama.org/environments/box2d/lunar_lander/
- Hyper parameter and abstract DQN: https://www.researchgate.net/publication/333145451_Deep_Q-Learning_on_Lunar_Lander_Game
- Implement DQN with Lunar Lander v2: https://github.com/yuchen071/DQN-for-LunarLander-v2
- Implement Double DQN: https://docs.agilerl.com/en/stable/tutorials/gymnasium/agilerl_td3_tutorial.html
